In [1]:
import pandas as pd
pd.options.display.max_columns = 100

### 데이터 추출
>   - 전월세 데이터 추출

In [4]:
# 전월세 csv 파일 불러오기
df_2024 = pd.read_csv('./data/서울특별시_전월세가_2024.csv', encoding='cp949')
df_2023 = pd.read_csv('./data/서울특별시_전월세가_2023.csv')

# 1인가구 면적 필터링
df_2024 = df_2024[df_2024['임대면적(㎡)'] <= 35].reset_index(drop=True).copy()
df_2023 = df_2023[df_2023['임대면적'] <= 35].reset_index(drop=True).copy()

# 임대료 0원 제거
df_2024 = df_2024[df_2024['임대료(만원)'] != 0].reset_index(drop=True).copy()
df_2023 = df_2023[df_2023['임대료(만원)'] != 0].reset_index(drop=True).copy()

# 불필요한 컬럼 제거
df_2024 = df_2024[['접수연도', '자치구명', '전월세 구분', '임대면적(㎡)', '보증금(만원)', '임대료(만원)', '건물용도', '신규갱신여부']].copy()
df_2023 = df_2023[['접수년도', '자치구명', '전월세구분', '임대면적', '보증금(만원)', '임대료(만원)', '건물용도', '신규계약구분']].copy()

# 컬럼명 전체 변경
df_2024.columns = [['접수년도', '자치구명', '전월세구분', '임대면적', '보증금', '임대료', '건물용도', '신규갱신구분']]
df_2023.columns = [['접수년도', '자치구명', '전월세구분', '임대면적', '보증금', '임대료', '건물용도', '신규갱신구분']]

# 결측값 제거
df_2024 = df_2024.dropna().reset_index(drop=True)
df_2023 = df_2023.dropna().reset_index(drop=True)

# 돈 단위 변경
df_2024['보증금'] = df_2024['보증금'].apply(lambda x: x * 10000)
df_2024['임대료'] = df_2024['임대료'].apply(lambda x: x * 10000)
df_2023['보증금'] = df_2023['보증금'].apply(lambda x: x * 10000)
df_2023['임대료'] = df_2023['임대료'].apply(lambda x: x * 10000)

# 데이터프레임 행으로 합치기
merge_df = pd.concat([df_2023, df_2024])

# index 초기화
merge_df.reset_index(drop=True, inplace=True)

>   - 청년 안심 주택 데이터 추출  


In [34]:
import requests
from bs4 import BeautifulSoup
url = "https://soco.seoul.go.kr/youth/pgm/home/yohome/maplist.json"
head = {"User-Agent":
"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"}


response = requests.post(url, headers= head)

ansim = response.json()
ansim_df = pd.DataFrame(ansim['mapResultList'])

select_col = ['adresGu','adresRo', 'scaleTot','roomTotArea','moneyRentalLow','moneyDepositLow',
       'fnMoneyDepositLow', 'fnMoneyDepositHigh', 'fnMoneyRentalLow',
       'fnMoneyRentalHigh']

ansim2 = ansim_df[select_col]
a = ansim2['roomTotArea'].tolist()
ansim2.drop(14, axis=0,inplace = True)
c = ansim2['fnMoneyDepositLow'].tolist()
d = ansim2['fnMoneyDepositHigh'].tolist()


import re

# 주어진 문자열 리스트
strings = ['4천만', '4천3백만', '3천6백만', '1억9천1백만']

# 정규식 패턴 정의
pattern = re.compile(r'(\d+)(천|백|만|억)?')

# 문자열을 숫자로 변환하는 함수
def convert_to_number(word):
    number = 0
    matches = pattern.findall(word)
    for match in matches:
        digit = int(match[0])
        unit = match[1]
        if unit == '천':
            digit *= 10000000
        elif unit == '백':
            digit *= 1000000
        elif unit == '만':
            digit *= 10000
        elif unit == '억':
            digit *= 100000000  # 억 단위는 1억 = 1e8
        else:
            digit *= 1
        number += digit
    return number

# 각 문자열을 숫자로 변환하여 리스트 생성
new_list = [convert_to_number(word) for word in c]
new_list2 = [convert_to_number(word) for word in d]

# 결과 출력
print(new_list)


>   - LH & SH 데이터 추출

In [38]:
import pandas as pd

mae = pd.read_csv('./data/한국토지주택공사_(매입임대)마이홈포털 기존임대주택 목록_20240104.csv',encoding='cp949') # 매입임대에 대한 csv파일
hang = pd.read_csv('./data/한국토지주택공사_(행복주택)마이홈포털 기존임대주택 목록_20240104.csv', encoding='cp949') # 행복주택에 대한 csv파일
mae = mae[mae['광역시도'] == "서울특별시"]
hang = hang[hang['광역시도'] == "서울특별시"]
mae = mae[mae['공급면적(전용)'] <= float(40)] # 전용면적에 대한 제한으로 청년주택의 주 대상인 1인 가구 추출을 위해 적용
hang = hang[hang['공급면적(전용)'] <= float(40)]

mae1=mae[~mae.도로명주소.duplicated()].drop(['광역시도','단지명','임대사업자','주차수','형명','공급면적(전용)','공급면적(공용)','세대수',\
                                      '임대보증금','월임대료','전환보증금'],axis=True).copy()
hang1=hang[~hang.도로명주소.duplicated()].drop(['광역시도','단지명','임대사업자','주차수','형명','공급면적(전용)','공급면적(공용)','세대수',\
                                      '임대보증금','월임대료','전환보증금','준공일자','건물형태','난방방식','승강기설치여부'],axis=True).copy()
""" 중복 데이터 제거 후 평균값'들'을 합치기 위한 전처리 """

mae2 = mae.groupby('도로명주소',as_index=False).mean('월임대료')
mae2 = mae2.drop(['공급면적(공용)','주차수','전환보증금'],axis=True)
hang2 = hang.groupby('도로명주소',as_index=False).mean('월임대료')
hang2 = hang2.drop(['공급면적(공용)','주차수','전환보증금'],axis=True) #도로명 주소로 그룹화 한 후 평균값 구하고 합치기 위한 불필요 column 제거

mae3 = pd.merge(mae1,mae2,how="inner",on='도로명주소')
hang3 = pd.merge(hang1,hang2,how="inner",on='도로명주소') #합체

for x in mae3.columns[4:] :
    mae3 = mae3.astype({x:'int'})
for y in hang3.columns[4:] :
    hang3 = hang3.astype({y:'int'}) # 세대수, 전용면적, 보증금, 임대료에 대한 타입 변경

mae3.rename(columns={'임대종류':'kindRental','시군구':'adresGu', '도로명주소':'adresWay', '주택유형':'kindHouse', '세대수':'scaleTot', \
             '공급면적(전용)':'totarea', '임대보증금':'depositMoney', '월임대료':'rentalMoney'},inplace=True)
hang3.rename(columns={'임대종류':'kindRental','시군구':'adresGu', '도로명주소':'adresWay', '주택유형':'kindHouse', '세대수':'scaleTot', \
             '공급면적(전용)':'totarea', '임대보증금':'depositMoney', '월임대료':'rentalMoney'},inplace=True) #column명 변경

allLh=pd.concat([mae3,hang3])

allLh=allLh[['kindRental', 'adresGu', 'adresWay','scaleTot', 'kindHouse', 'totarea',\
       'depositMoney', 'rentalMoney']] #column 순서 변경 scaleTot <-> kindHouse

allLh = allLh.where(pd.notnull(allLh), "알수없음") #결측치 변경


/tmp/ipykernel_7059/74720895.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lhlist = pd.read_csv("./data/한국토지주택공사_(매입임대)마이홈포털 기존임대주택 목록_20240104.csv", encoding='cp949')
/tmp/ipykernel_7059/74720895.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ansim2.drop(14, axis=0,inplace = True)


### DB 접속
>   - DB 정보 불러오기

In [22]:
with open("./db.config",'r') as f:
    db_data = f.read()
db_data = eval(db_data)

In [26]:
import pymysql

_host = db_data['DB_HOST']
_user = db_data['DB_USER']
_password = db_data['DB_PASSWORD']
_name = db_data['DB_NAME']

con = pymysql.connect(host=f'{_host}', user=f'{_user}', password=f'{_password}', db=f'{_name}', charset='utf8')
cur = con.cursor()

### 데이터 전송
>   - allRent 테이블 생성: 서울 전지역

In [42]:
sql = 'insert into allRent values (%s, %s, %s, %s, %s, %s, %s, %s)'
from tqdm import tqdm

for idx, row in tqdm(merge_df.iterrows()):
    cur.execute(sql, row.tolist())

con.commit()

>   - allLh 테이블 생성: LH & SH

In [ ]:
sql = "INSERT INTO allLh VALUES (%s, %s, %s, %s, %s, %s, %s, %s)" 

allLh.apply(lambda x : cur.execute(sql,x.tolist()),axis=1) #데이터 전송

con.commit()

In [27]:
# DB 데이터 불러오기``
lh_df = pd.read_sql_query('select * from allLh', con=con)
ansim_df = pd.read_sql_query('select * from ansim', con=con)

/tmp/ipykernel_7059/50531298.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lh_df = pd.read_sql_query('select * from allLh', con=con)
/tmp/ipykernel_7059/50531298.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ansim_df = pd.read_sql_query('select * from ansim', con=con)
